In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
import re
import sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pickle


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saleen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
sms = pd.read_csv('spam.csv',encoding='Windows-1252')
# Dropping the redundent looking collumns (for this project)
to_drop = ["Unnamed: 2","Unnamed: 3","Unnamed: 4"]
sms = sms.drop(sms[to_drop], axis=1)
# Renaming the columns because I feel fancy today 
sms.rename(columns = {"v1":"label", "v2":"message"}, inplace = True)
# ham_rows = sms[sms['label'] == 'ham']
# num_rows_to_delete = int(0.54 * len(ham_rows))
# rows_to_delete = np.random.choice(ham_rows.index, num_rows_to_delete, replace=False)
# sms.drop(rows_to_delete, inplace=True)
sms.head()
corpus = []
ps = PorterStemmer()

In [5]:
sms.head()
print(sms.shape[0])
for i in range(0,sms.shape[0]):
    message = re.sub(pattern=r'[^a-zA-Z]', repl=' ', string=str(sms.message[i])) #Cleaning special character from the message
    # print(message)
    message = message.lower() #Converting the entire message into lower case
    # print(message)
    words = message.split() # Tokenizing the review by words
    # print(words)
    words = [word for word in words if word not in set(stopwords.words('english'))] #Removing the stop words
    # print(words)
    words = [ps.stem(word) for word in words] #Stemming the words
    # print(words)
    message = ' '.join(words) #Joining the stemmed words
    # print(message)
    corpus.append(message) #Building a corpus of messages

5572


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sms["label"])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [8]:
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(y_pred)
acc_s = accuracy_score(y_test, y_pred)*100
print("Accuracy Score {} %".format(round(acc_s,2)))

[0 0 0 ... 0 0 0]
Accuracy Score 98.65 %


In [9]:
# Save the trained model and the preprocessed data to disk
with open(r'sms\archive\pickle\sms_model2.pkl', 'wb') as f:
    pickle.dump(classifier, f)
with open(r'sms\archive\pickle\preprocessed_data.pkl', 'wb') as f:
    pickle.dump((cv, X, y), f)